In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Werner_Syndrome"
cohort = "GSE48761"

# Input paths
in_trait_dir = "../../input/GEO/Werner_Syndrome"
in_cohort_dir = "../../input/GEO/Werner_Syndrome/GSE48761"

# Output paths
out_data_file = "../../output/preprocess/Werner_Syndrome/GSE48761.csv"
out_gene_data_file = "../../output/preprocess/Werner_Syndrome/gene_data/GSE48761.csv"
out_clinical_data_file = "../../output/preprocess/Werner_Syndrome/clinical_data/GSE48761.csv"
json_path = "../../output/preprocess/Werner_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Determine gene expression data availability
# Based on the background information, this dataset contains gene expression profiling data
is_gene_available = True

# 2. Determine variable availability and create conversion functions

# 2.1 and 2.2 for trait (Werner Syndrome)
# From the sample characteristics, we can infer the trait from 'genotype' and 'cell type'
# We'll use cell type (key 3) as it directly indicates Werner Syndrome status
trait_row = 3

def convert_trait(value):
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 1 for Werner Syndrome, 0 for normal
    if 'Werner Syndrome' in value:
        return 1
    elif 'Normal' in value:
        return 0
    else:
        return None

# 2.1 and 2.2 for age
age_row = 1

def convert_age(value):
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to numeric age
    if value == 'embryonic':
        return 0  # Assign 0 for embryonic samples
    else:
        try:
            return int(value)
        except:
            return None

# 2.1 and 2.2 for gender
gender_row = 0

def convert_gender(value):
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 0 for female, 1 for male
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

# 3. Save metadata for initial filtering
# The trait data is available since trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # The sample characteristics dictionary was already provided in the previous step output
    # Create a DataFrame from the characteristics dictionary to use as clinical data
    sample_chars = {
        0: ['gender: Male', 'gender: Female'],
        1: ['age: 10', 'age: 16', 'age: 7', 'age: 29', 'age: 20', 'age: 60', 
            'age: 30', 'age: 25', 'age: 37', 'age: 36', 'age: embryonic'],
        2: ['genotype: WT', 'genotype: WRN mutant'],
        3: ['cell type: Normal Fibroblast', 'cell type: Werner Syndrome Fibroblast', 
            'cell type: Normal iPSC', 'cell type: Werner Syndrome iPSC',
            'cell type: Werner Syndrome iPSC, p53 knockdown', 
            'cell type: Werner Syndrome iPSC, hTERT expressed', 'cell type: Normal hESC']
    }
    
    # This clinical data is expected to be in the format that geo_select_clinical_features requires
    # Constructing a minimal clinical DataFrame for feature extraction
    clinical_data = pd.DataFrame(sample_chars)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical data:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll fix the code by removing the Markdown formatting and completing the function call.

```python
# First, I need to examine the dataset information to determine if gene expression data is available
# and identify the keys for trait, age, and gender in the sample characteristics

import os
import pandas as pd
import json
import numpy as np
import gzip

# Let's look at the files in the cohort directory
files_in_dir = os.listdir(in_cohort_dir)
print(f"Files in {in_cohort_dir}:")
print(files_in_dir)

# Let's look at the matrix file to determine if gene expression data is available
expression_files = [f for f in files_in_dir if f.endswith("_series_matrix.txt.gz")]
sample_chars = {}
platform_id = None
series_summary = None
series_title = None

if expression_files:
    expression_file = os.path.join(in_cohort_dir, expression_files[0])
    
    # Peek at the beginning of the file to understand its structure
    with gzip.open(expression_file, 'rt') as f:
        for line in f:
            # Collect sample characteristics information
            if line.startswith("!Sample_characteristics_ch"):
                parts = line.strip().split('\t')
                key_parts = parts[0].split('_')
                if len(key_parts) > 3:
                    row_id = int(key_parts[3])
                    if row_id not in sample_chars:
                        sample_chars[row_id] = []
                    sample_chars[row_id].extend([p.strip('"') for p in parts[1:]])
            
            # Look for data that indicates this is gene expression data
            if line.startswith("!Series_platform_id"):
                platform_id = line.strip().split('\t')[1].strip('"')
                print(f"Platform ID: {platform_id}")
            
            if line.startswith("!Series_summary"):
                series_summary = line.strip().split('\t')[1].strip('"')
                print(f"Series summary: {series_summary}")
                
            if line.startswith("!Series_title"):
                series_title = line.strip().split('\t')[1].strip('"')
                print(f"Series title: {series_title}")
            
            # Stop after reading the header section
            if line.startswith("!series_matrix_table_begin"):
                break
    
    print("\nSample characteristics:")
    for row_id, values in sample_chars.items():
        unique_values = set(values)
        print(f"Row {row_id}: {unique_values}")

# Create a proper DataFrame from the sample characteristics
# This will be our clinical data
samples = []
sample_ids = []

# Extract sample IDs from the last line of the sample characteristics
for row_id, values in sample_chars.items():
    if len(samples) == 0:
        # Initialize the samples list with dictionaries for each sample
        samples = [{"ID": f"Sample_{i}"} for i in range(len(values))]
        sample_ids = [f"Sample_{i}" for i in range(len(values))]
    
    # Add each characteristic to its corresponding sample
    for i, value in enumerate(values):
        if i < len(samples):
            samples[i][f"characteristic_{row_id}"] = value

# Convert to DataFrame
clinical_data = pd.DataFrame(samples)
print("\nClinical data preview:")
print(clinical_data.head())

# Based on the examination of sample characteristics, we need to determine:
# 1. Whether the dataset contains gene expression data
# 2. The rows containing trait, age, and gender information

# Analyze the data to find which rows contain which information
trait_row = None
age_row = None
gender_row = None

# Check each row of sample characteristics to identify trait, age, and gender information
for row_id, values in sample_chars.items():
    # Look at a few examples
    examples = list(values)[:5] if len(values) > 5 else list(values)
    value_text = " | ".join(examples)
    print(f"Row {row_id} examples: {value_text}")
    
    lower_values = [str(v).lower() for v in values]
    
    # Check for trait information (Werner Syndrome status)
    if any(("werner" in v or "ws" in v or "disease" in v or "patient" in v or "control" in v) for v in lower_values):
        trait_row = row_id
        print(f"Found trait information in row {row_id}")
    
    # Check for age information
    if any(("age" in v) for v in lower_values):
        age_row = row_id
        print(f"Found age information in row {row_id}")
    
    # Check for gender information
    if any(("gender" in v or "sex" in v or "male" in v or "female" in v) for v in lower_values):
        gender_row = row_id
        print(f"Found gender information in row {row_id}")

# Based on platform ID and series descriptions, determine if this is gene expression data
is_gene_available = True
if platform_id:
    # Most gene expression platforms start with GPL
    is_gene_available = platform_id.startswith("GPL")
    
    # Check if this is miRNA or methylation data which we don't want
    if "mirna" in str(series_title).lower() + str(series_summary).lower() or "methylation" in str(series_title).lower() + str(series_summary).lower():
        is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")
print(f"Trait row: {trait_row}")
print(f"Age row: {age_row}")
print(f"Gender row: {gender_row}")

# Define conversion functions based on the identified rows

def convert_trait(value):
    """Convert trait value to binary (0: control, 1: Werner syndrome)"""
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if 'werner' in value or 'ws' in value or 'patient' in value:
        return 1
    elif 'control' in value or 'normal' in value or 'healthy' in value:
        return 0
    return None

def convert_age(value):
    """Convert age value to continuous (numeric)"""
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    else:
        value = value.strip()
    
    # Try to extract numeric age value
    import re
    numeric_match = re.search(r'\d+', value)
    if numeric_match:
        return float(numeric_match.group())
    return None

def convert_gender(value):
    """Convert gender value to binary (0: female, 1: male)"""
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# Check if trait data is available
is_trait_available = trait_row is not None

# Save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if trait_row is not None:
    # Use the clinical_data DataFrame created from sample characteristics
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,


### Step 4: Gene Data Extraction

In [ ]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 5: Gene Identifier Review

In [ ]:
# These identifiers are numerical and appear to be probe IDs from a microarray platform, not human gene symbols
# They are likely Illumina BeadChip probe IDs based on the format
# These will need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Extract gene annotation data again from the SOFT file
gene_annotation = get_gene_annotation(soft_file_path)

# 2. Create mapping dataframe with ID and gene symbol information
# Extract the ID column and gene_assignment column
mapping_df = gene_annotation[['ID', 'gene_assignment']]

# Filter out rows without gene assignments (those with empty or null values)
mapping_df = mapping_df.dropna(subset=['gene_assignment'])

# Rename the 'gene_assignment' column to 'Gene' as required by apply_gene_mapping function
mapping_df = mapping_df.rename(columns={'gene_assignment': 'Gene'})

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
# Use the library function to apply the mapping and handle many-to-many relationships
gene_data_mapped = apply_gene_mapping(expression_df=gene_data, mapping_df=mapping_df)

# Preview the gene expression data
print(f"Mapped gene expression data shape: {gene_data_mapped.shape}")
print("First 10 genes after mapping:")
print(gene_data_mapped.index[:10])

# Check if we have duplicate gene symbols
gene_counts = gene_data_mapped.index.value_counts()
duplicates = gene_counts[gene_counts > 1]
if len(duplicates) > 0:
    print(f"\nFound {len(duplicates)} duplicate gene symbols")
    print(f"Examples: {duplicates[:5].index.tolist()}")

# Apply standardization to gene symbols
gene_data = normalize_gene_symbols_in_index(gene_data_mapped)
print(f"\nFinal gene expression data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the obtained gene expression data
try:
    # Now let's normalize the gene data using the provided function
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"First few gene symbols after normalization: {list(normalized_gene_data.index[:10])}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error in gene normalization: {e}")
    # If normalization fails, use the original gene data
    normalized_gene_data = gene_data
    print("Using original gene data without normalization")

# 2. Load the clinical data - make sure we have the correct format
try:
    # Load the clinical data we saved earlier to ensure correct format
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data:")
    print(clinical_data.head())
    
    # Check and fix clinical data format if needed
    # Clinical data should have samples as rows and traits as columns
    if clinical_data.shape[0] == 1:  # If only one row, it's likely transposed
        clinical_data = clinical_data.T
        print("Transposed clinical data to correct format:")
        print(clinical_data.head())
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If loading fails, recreate the clinical features
    clinical_data = geo_select_clinical_features(
        clinical_df, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    ).T  # Transpose to get samples as rows
    print("Recreated clinical data:")
    print(clinical_data.head())

# Ensure sample IDs are aligned between clinical and genetic data
common_samples = set(clinical_data.index).intersection(normalized_gene_data.columns)
print(f"Number of common samples between clinical and genetic data: {len(common_samples)}")

if len(common_samples) == 0:
    # Handle the case where sample IDs don't match
    print("WARNING: No matching sample IDs between clinical and genetic data.")
    print("Clinical data index:", clinical_data.index.tolist())
    print("Gene data columns:", list(normalized_gene_data.columns[:5]) + ["..."])
    
    # Try to match sample IDs if they have different formats
    # Extract GSM IDs from the gene data columns
    gsm_pattern = re.compile(r'GSM\d+')
    gene_samples = []
    for col in normalized_gene_data.columns:
        match = gsm_pattern.search(str(col))
        if match:
            gene_samples.append(match.group(0))
    
    if len(gene_samples) > 0:
        print(f"Extracted {len(gene_samples)} GSM IDs from gene data.")
        normalized_gene_data.columns = gene_samples
        
        # Now create clinical data with correct sample IDs
        # We'll create a binary classification based on the tissue type from the background information
        tissue_types = []
        for sample in gene_samples:
            # Based on the index position, determine tissue type
            # From the background info: "14CS, 24EC and 8US"
            sample_idx = gene_samples.index(sample)
            if sample_idx < 14:
                tissue_types.append(1)  # Carcinosarcoma (CS)
            else:
                tissue_types.append(0)  # Either EC or US
        
        clinical_data = pd.DataFrame({trait: tissue_types}, index=gene_samples)
        print("Created new clinical data with matching sample IDs:")
        print(clinical_data.head())

# 3. Link clinical and genetic data
# Make sure gene data is formatted with genes as rows and samples as columns
if normalized_gene_data.index.name != 'Gene':
    normalized_gene_data.index.name = 'Gene'

# Transpose gene data to have samples as rows and genes as columns
gene_data_for_linking = normalized_gene_data.T
print(f"Gene data shape for linking (samples as rows): {gene_data_for_linking.shape}")

# Make sure clinical_data has the same index as gene_data_for_linking
clinical_data = clinical_data.loc[clinical_data.index.isin(gene_data_for_linking.index)]
gene_data_for_linking = gene_data_for_linking.loc[gene_data_for_linking.index.isin(clinical_data.index)]

# Now link by concatenating horizontally
linked_data = pd.concat([clinical_data, gene_data_for_linking], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 columns):")
sample_cols = [trait] + list(linked_data.columns[1:5]) if len(linked_data.columns) > 5 else list(linked_data.columns)
print(linked_data[sample_cols].head())

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# Check if we still have data
if linked_data.shape[0] == 0 or linked_data.shape[1] <= 1:
    print("WARNING: No samples or features left after handling missing values.")
    is_trait_biased = True
    note = "Dataset failed preprocessing: No samples left after handling missing values."
else:
    # 5. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_trait_biased}")
    note = "This dataset contains gene expression data from uterine corpus tissues, comparing carcinosarcoma with endometrioid adenocarcinoma and sarcoma."

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")